<h1>Perform Federated Learning at Grid Platform </h1>
In this notebook, we will train a model using federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Alice (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

In [3]:
import syft as sy
from syft.grid.public_grid import PublicGridNetwork
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
hook = sy.TorchHook(th)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


device = th.device("cuda:0" if th.cuda.is_available() else "cpu")

if(th.cuda.is_available()):
    th.set_default_tensor_type(th.cuda.FloatTensor)
    
model = Net()
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [6]:
GRID_ADDRESS = 'localhost'
GRID_PORT = '5000'

my_grid = PublicGridNetwork(hook,"http://" + GRID_ADDRESS + ":" + GRID_PORT)

In [7]:
data = my_grid.search("#X", "#mnist", "#dataset")
target = my_grid.search("#Y", "#mnist", "#dataset")

In [8]:
data

{'Bob': [(Wrapper)>[PointerTensor | me:69774305752 -> Bob:84790492900]
  	Tags: #mnist #X #dataset 
  	Shape: torch.Size([5000, 1, 28, 28])
  	Description: The input datapoints to the MNIST dataset....],
 'Alice': [(Wrapper)>[PointerTensor | me:26812650417 -> Alice:53510693506]
  	Tags: #mnist #X #dataset 
  	Shape: torch.Size([5000, 1, 28, 28])
  	Description: The input datapoints to the MNIST dataset....]}

In [9]:
target

{'Bob': [(Wrapper)>[PointerTensor | me:85263268143 -> Bob:96713703699]
  	Tags: #mnist #Y #dataset 
  	Shape: torch.Size([5000])
  	Description: The input labels to the MNIST dataset....],
 'Alice': [(Wrapper)>[PointerTensor | me:78099185844 -> Alice:47171429524]
  	Tags: #mnist #Y #dataset 
  	Shape: torch.Size([5000])
  	Description: The input labels to the MNIST dataset....]}

In [11]:
data = list(data.values())
target = list(target.values())

In [12]:
def epoch_total_size(data):
    total = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            total += data[i][j].shape[0]
            
    return total

In [14]:
N_EPOCS = 3
SAVE_MODEL = True
SAVE_MODEL_PATH = './models'

def train(epoch):
    model.train()
    epoch_total = epoch_total_size(data)
    current_epoch_size = 0
    for i in range(len(data)):
        for j in range(len(data[i])):
            current_epoch_size += len(data[i][j])
            worker = data[i][j].location
            model.send(worker)
            optimizer.zero_grad()
            pred = model(data[i][j])
            loss = criterion(pred, target[i][j])
            loss.backward()
            optimizer.step()
            model.get()
            loss = loss.get()
            print('Train Epoch: {} | With {} data |: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                      epoch, worker.id, current_epoch_size, epoch_total,
                            100. *  current_epoch_size / epoch_total, loss.item()))
                    
for epoch in range(N_EPOCS):
    train(epoch)

Train Epoch: 0 | With Bob data |: [5000/10000 (50%)]	Loss: 2.283551
Train Epoch: 0 | With Alice data |: [10000/10000 (100%)]	Loss: 2.279320
Train Epoch: 1 | With Bob data |: [5000/10000 (50%)]	Loss: 2.274262


Websocket connection closed (worker: Alice)
Created new websocket connection


Train Epoch: 1 | With Alice data |: [10000/10000 (100%)]	Loss: 2.270052
Train Epoch: 2 | With Bob data |: [5000/10000 (50%)]	Loss: 2.265139
Train Epoch: 2 | With Alice data |: [10000/10000 (100%)]	Loss: 2.260950
